In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """
    # 네이버 금융 -> 시세 -> 개발자 도구 -> Networks -> Doc -> user-agent
    # 브라우저임을 알려줘야 한다
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}'
    
    response = requests.get(url, headers=headers)
    html = bs(response.text, 'lxml')
    table = html.select('table')
    temp = pd.read_html(str(table))
    return temp[0].dropna()

def get_item_list(item_code, item_name, page_no):
    """
    일자별 시세를 수집
    """
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        # 0.1초씩 쉬어간다
        time.sleep(0.1)
        # shape: (10, 7)
        # 행 10개 , 즉 데이터가 더 이상 없는 경우
        if temp.shape[0] < 10:
            break
     
    df = pd.concat(item_list)
    
    df['종목코드'] = item_code
    df['종목명'] = item_name

    # 순서 변경
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]
        
    return df

In [2]:
item_code = '323990'
item_name = '박셀바이오'
page_no = 1

get_item_list(item_code, item_name, page_no)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,323990,박셀바이오,2021.09.03,81800.0,200.0,82500.0,83500.0,81300.0,312425.0
2,323990,박셀바이오,2021.09.02,82000.0,2500.0,81000.0,83100.0,79000.0,482785.0
3,323990,박셀바이오,2021.09.01,79500.0,700.0,78800.0,81500.0,78800.0,220885.0
4,323990,박셀바이오,2021.08.31,78800.0,300.0,79100.0,81200.0,77500.0,365477.0
5,323990,박셀바이오,2021.08.30,79100.0,2200.0,78000.0,81500.0,77000.0,594635.0
...,...,...,...,...,...,...,...,...,...
2,323990,박셀바이오,2020.09.28,28000.0,1050.0,27800.0,29150.0,27050.0,1628177.0
3,323990,박셀바이오,2020.09.25,26950.0,450.0,27100.0,30500.0,26000.0,4612185.0
4,323990,박셀바이오,2020.09.24,26500.0,1000.0,24450.0,31200.0,24000.0,8325838.0
5,323990,박셀바이오,2020.09.23,25500.0,4200.0,22150.0,27650.0,22050.0,8921196.0
